In [ ]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_data = []
with open('/home/data/train.json', 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# Convert to DataFrame for easier analysis
train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {train_df.columns.tolist()}")
print("\nFirst few rows:")
train_df.head()

In [ ]:
# Check target distribution
target_col = 'requester_received_pizza'
print("Target distribution:")
print(train_df[target_col].value_counts())
print(f"\nPercentage distribution:")
print(train_df[target_col].value_counts(normalize=True) * 100)

# Check for missing values
print(f"\nMissing values:")
print(train_df.isnull().sum().sum())